In [ ]:
#------------------- Change working directory to project root -------------------#
from pathlib import Path, os

cur = Path().resolve()
while not (cur / "src").is_dir():
    if cur == cur.parent: raise RuntimeError("No 'src' dir")
    cur = cur.parent

os.chdir(cur)
print(f"[INFO] Changed working directory to project root: {cur}")

In [ ]:
# -------------------------------------- External Libraries --------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import defaultdict
import time
# ----------------------------------------- Local Imports ----------------------------------------
from src.core                  import LoRaPhyParams, LoRaFrameParams
from src.mod                   import LoRaModulator
from src.demod                 import LoRaDemodulator
from src.sync                  import DechirpBasedSynchronizer
from src.core.snr_utils        import estimate_snr_from_ls_fit, SDRProfile, AttenSNRPoint
from src.core.vpn_utils        import VPNKeepAlive
import src.core.sdr_utils as sdr_utils



In [ ]:
def plot_snr_map(profile:SDRProfile):
    """
    Plot the SNR map from the given SDR profile.
    """
    atten_range = [point.attenuation for point in profile.snr_map]
    avg_snr = [np.mean(point.snr_values) for point in profile.snr_map]

    plt.figure(figsize=(10, 5))
    plt.plot(atten_range, avg_snr, marker='o')
    plt.title(f"SNR Map: {profile.name}")
    plt.xlabel("Tx Attenuation (dB)")
    plt.ylabel("Estimated SNR (dB)")
    plt.grid(True)
    plt.xticks(atten_range)
    plt.tight_layout()
    plt.show()


In [ ]:
def run_calibration(profile_name, atten_range, target_successful_samples, max_retries_per_atten, sdr_params:sdr_utils.SDRParams, modulator:LoRaModulator, demodulator:LoRaDemodulator, synchronizer:DechirpBasedSynchronizer, fold_mode):
    """
    Run the SNR calibration process.
    
    :param target_successful_samples: Number of successful samples to collect per attenuation level.
    :param max_retries_per_atten: Maximum number of attempts per attenuation level.
    :param sdr_params: SDR parameters for the calibration.
    :param modulator: LoRa modulator instance.
    :param demodulator: LoRa demodulator instance.
    :param synchronizer: Synchronizer instance for frame synchronization.
    :return: A SDRProfile object containing the results of the calibration.
    """

    phy_params = modulator.phy_params
    frame_params = modulator.frame_params

    snr_map = defaultdict(list)

    payload_symbols = sdr_utils.optimize_payload_symbols(
        sdr_params.rx_buffer_size,
        frame_params.preamble_symbol_count,
        phy_params.samples_per_symbol,
        pad_samples=0
    )
    rng = np.random.default_rng(23)
    payload = list(rng.integers(0, phy_params.chips_per_symbol, size=payload_symbols))

    modulated_full = modulator.modulate(payload, include_frame=True)
    reference_payload = modulator.modulate(payload, include_frame=False)

    vpn_guard = VPNKeepAlive() # Mantiene la conexión VPN activa durante la calibración, ya que esta puede durar un tiempo largo
    vpn_guard.reconnect()  # Asegurarse de que la VPN está conectada antes de comenzar
    sdr = sdr_utils.init_sdr(sdr_params)  
    del sdr # resetear el SDR para evitar problemas
    
    # Corrida de atenuación
    for idx, atten in enumerate(atten_range):

        vpn_guard.maybe_reconnect()  # Reconexión périódica de la VPN para evitar fallos de rekey

        print(f"[SNR] Probando TxAtten = {atten} dB ({idx + 1}/{len(atten_range)})...")

        

        success_cnt = 0
        attempt_cnt = 0

        while attempt_cnt < max_retries_per_atten and success_cnt < target_successful_samples:
            attempt_cnt += 1
            demodulator.backend.clear_memory()

            try:
                # ---------- TX / RX ----------
                sdr = sdr_utils.init_sdr(sdr_params)
                sdr_utils.change_sdr_attenuation(sdr, atten)
                time.sleep(0.1) 
                sdr.tx(modulated_full * (2**15 - 1))
                time.sleep(0.5)
                _ = sdr.rx()
                iq_buffer = sdr.rx() / (2**15 - 1)
                sdr_utils.soft_delete_sdr(sdr)

                # ---------- SYNC / DEMOD ----------
                aligned = synchronizer.run(iq_buffer)
                symbols = demodulator.demodulate(aligned)

                symbols = symbols.get() if hasattr(symbols, "get") else symbols
        
                if len(symbols) != len(payload):
                    raise RuntimeError("Las longitudes no coinciden")

                snr = estimate_snr_from_ls_fit(aligned, reference_payload)
                print(f"  ✓ intento {attempt_cnt}: SNR {snr:.2f} dB")

                snr_map[atten].append(snr)
                success_cnt += 1

            except Exception as e:
                print(f"  ✗ intento {attempt_cnt}: {e}")
                if "No device found" in str(e):
                    raise

        # ------ después del bucle ------

        if success_cnt == 0:
            print(f"  → no valid samples at {atten} dB (skipped)")
        else:
            print(f"  → kept {success_cnt} sample(s) at {atten} dB")
    
    # COnstruir el perfil SDR
    print("\n[INFO] Construyendo el perfil SDR...")
    atten_snr_points = [
        AttenSNRPoint(attenuation=att, snr_values=snrs)
        for att, snrs in sorted(snr_map.items())
    ]

    profile = SDRProfile(
        name=profile_name,
        fold_mode=fold_mode,
        tx_sdr_params=sdr_params,
        rx_sdr_params=sdr_params,
        phy_params=phy_params,
        frame_params=frame_params,
        snr_map=atten_snr_points
    )

    vpn_guard.disconnect()  # Desconectar VPN

    return profile




In [ ]:
# --- Notebook Configuration ------------------------------------------
modulator_backend = "numpy"    
demodulator_backend = "cupy"   
fold_mode = "0FPA"     

phy_params = LoRaPhyParams(
    spreading_factor=8,
    bandwidth=500e3,
    samples_per_chip= 2
)

frame_params = LoRaFrameParams(
    preamble_symbol_count=8,
    explicit_header=True,
    sync_word=0x00
)

sdr_params = sdr_utils.SDRParams(
    uri = "ip:192.168.1.34",
    sample_rate = int(phy_params.sample_rate) ,
    lo_freq = 938e6,
    loopback_mode = sdr_utils.LoopbackMode.OTA,
    rf_bandwidth = int(phy_params.bandwidth)* 2,
    tx_attenuation = -29, # Esto cambiará
    rx_gain_control=sdr_utils.RxGainControl(sdr_utils.RxGainControlMode.MANUAL, 73),
    rx_buffer_size= 2**20
)

In [ ]:

modulator = LoRaModulator(
    phy_params, 
    frame_params, 
    backend=modulator_backend
)

demodulator = LoRaDemodulator(
    phy_params,
    backend=demodulator_backend,
    fold_mode=fold_mode
)
synchronizer = DechirpBasedSynchronizer(
    phy_params,
    frame_params,
    backend=demodulator_backend,
    fold_mode=fold_mode,
    max_sync_candidates=50
)

In [ ]:
payload_symbols = sdr_utils.optimize_payload_symbols(
    sdr_params.rx_buffer_size,
    frame_params.preamble_symbol_count,
    phy_params.samples_per_symbol,
    pad_samples=0
)
print(f"[INFO] Payload symbols per frame: {payload_symbols}")
print(f"[INFO] (limit: {1<<(2*phy_params.spreading_factor)}")

In [ ]:
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

profile = run_calibration(
    profile_name=f"[LoRa-SDR]-SF8-500kHz-2SPC-0FPA",
    atten_range=list(range(-55, -83, -1)),
    target_successful_samples=20,
    max_retries_per_atten=40,
    sdr_params=sdr_params,
    modulator=modulator,
    demodulator=demodulator,
    synchronizer=synchronizer,
    fold_mode=fold_mode
)

plot_snr_map(profile)
profile.save(f"lora_sdr/sf8_500k_2spc_0fpa.json")